<a href="https://colab.research.google.com/github/Jonipeloni/musicgeneration/blob/main/Transformer_aktuell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pretty_midi numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.9 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592289 sha256=1b066391567b70a656e93a3ac28cc8071f784eff44874a7be42bcfb489fc2570
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [12]:
pip install miditok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.6 MB/s eta 0:00:00


In [13]:
import os
import shutil
import numpy as np
import tensorflow as tf
import pretty_midi

import collections
#import datetime
#import fluidsynth
#import glob
#import numpy as np
import pathlib
#import pandas as pd
#import pretty_midi
#import seaborn as sns
#import tensorflow as tf
from miditok import REMI

#from IPython import display
#from matplotlib import pyplot as plt
#from typing import Optional

In [4]:
#Loading Data
data_dir = pathlib.Path('data/maestro-v2.0.0')
if not data_dir.exists():
  tf.keras.utils.get_file(
      'maestro-v2.0.0-midi.zip',
      origin='https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip',
      extract=True,
      cache_dir='.', cache_subdir='data',
  )

59243107/59243107 [==============================] - 1s 0us/step


In [7]:
#Shrinking amount of Files
import shutil
from pathlib import Path

subdirectories = ["2006", "2008", "2009", "2011", "2013", "2014", "2015", "2017", "2018"]
for a in subdirectories:

  directory_to_delete = data_dir / a

  # Delete the directory and all its contents
  try:
      shutil.rmtree(directory_to_delete)
      print(f"Deleted {directory_to_delete}")
  except Exception as e:
      print(f"Error deleting {directory_to_delete}: {e}")


Error deleting data/maestro-v2.0.0/2006: [Errno 2] No such file or directory: 'data/maestro-v2.0.0/2006'
Deleted data/maestro-v2.0.0/2008
Deleted data/maestro-v2.0.0/2009
Deleted data/maestro-v2.0.0/2011
Deleted data/maestro-v2.0.0/2013
Deleted data/maestro-v2.0.0/2014
Deleted data/maestro-v2.0.0/2015
Deleted data/maestro-v2.0.0/2017
Deleted data/maestro-v2.0.0/2018


In [8]:
folders = ["2004"]
for folder in folders:
    folder_path = os.path.join(data_dir, folder)

    # Check if the folder exists
    if os.path.exists(folder_path) and os.path.isdir(folder_path):
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)

            # Check if it is a file and not a sub-directory
            if os.path.isfile(file_path):
                # Move file
                shutil.move(file_path, data_dir)

        # Remove the now-empty folder
        os.rmdir(folder_path)

In [10]:
# The path to the directory containing your MIDI files
midi_files_dir = data_dir

# Criteria for selection
desired_time_signature = (4, 4)  # (numerator, denominator)
min_length_seconds = 100
max_length_seconds = 800

selected_files = []

# Iterate over all files in the directory
for filename in os.listdir(midi_files_dir):
    if filename.endswith('.mid') or filename.endswith('.midi'):  # Check if it's a MIDI file
        file_path = os.path.join(midi_files_dir, filename)

        try:
            # Load the MIDI file
            midi_data = pretty_midi.PrettyMIDI(file_path)

            # Check time signatures
            time_signatures = midi_data.time_signature_changes
            has_desired_time_signature = any(ts.numerator == desired_time_signature[0] and
                                             ts.denominator == desired_time_signature[1]
                                             for ts in time_signatures)

            # Check length
            length = midi_data.get_end_time()  # This returns the length in seconds
            if has_desired_time_signature and min_length_seconds <= length <= max_length_seconds:
                selected_files.append(file_path)

        except Exception as e:
            print(f"Error processing {filename}: {e}")

# Print or use the selected files
for file in selected_files:
    print(file)


data/maestro-v2.0.0/MIDI-Unprocessed_08_R1_2006_01-04_ORIG_MID--AUDIO_08_R1_2006_Disk1_02_Track02_wav.midi
data/maestro-v2.0.0/MIDI-Unprocessed_XP_19_R2_2004_01_ORIG_MID--AUDIO_19_R2_2004_03_Track03_wav.midi
data/maestro-v2.0.0/MIDI-Unprocessed_12_R1_2006_01-08_ORIG_MID--AUDIO_12_R1_2006_04_Track04_wav.midi
data/maestro-v2.0.0/MIDI-Unprocessed_SMF_02_R1_2004_01-05_ORIG_MID--AUDIO_02_R1_2004_08_Track08_wav.midi
data/maestro-v2.0.0/MIDI-Unprocessed_XP_15_R2_2004_01_ORIG_MID--AUDIO_15_R2_2004_03_Track03_wav.midi
data/maestro-v2.0.0/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MID--AUDIO_17_R1_2006_04_Track04_wav.midi
data/maestro-v2.0.0/MIDI-Unprocessed_XP_22_R2_2004_01_ORIG_MID--AUDIO_22_R2_2004_02_Track02_wav.midi
data/maestro-v2.0.0/MIDI-Unprocessed_XP_14_R1_2004_04_ORIG_MID--AUDIO_14_R1_2004_04_Track04_wav.midi
data/maestro-v2.0.0/MIDI-Unprocessed_XP_09_R1_2004_01-02_ORIG_MID--AUDIO_09_R1_2004_01_Track01_wav.midi
data/maestro-v2.0.0/MIDI-Unprocessed_SMF_17_R1_2004_04_ORIG_MID--AUDIO_17_R1_2

In [37]:
from miditok import REMI
from pathlib import Path

# A validation method to discard MIDIs we do not want
def midi_valid(midi) -> bool:
    if any(ts.numerator != 4 for ts in midi.time_signature_changes):
        return False  # time signature different from 4/*, 4 beats per bar
    if midi.max_tick < 10 * midi.ticks_per_beat:
        return False  # this MIDI is too short
    return True

# Assume data_dir is defined somewhere in your code, e.g.,
# data_dir = Path("/path/to/your/data_dir")

# Define the folder path you want to create for tokens without using BPE
tokens_small_noBPE = data_dir / "tokens_small_noBPE"

# Create the folder
tokens_small_noBPE.mkdir(parents=True, exist_ok=True)

print(f"Created {tokens_small_noBPE}")

# Path to the MIDI files and path to where the tokens will be saved
path_midi_files = data_dir
path_tokens_noBPE = data_dir / 'tokens_small_noBPE/'

# Creates the tokenizer and lists the file paths
tokenizer = REMI()
midi_paths = selected_files # Make sure selected_files is defined and contains paths to your MIDI files

# Converts MIDI files to tokens saved as JSON files
tokenizer.tokenize_midi_dataset(
    midi_paths,
    Path(path_tokens_noBPE),
    midi_valid
)

# Removed steps:
# - Learning BPE from non-BPE tokens
# - Applying the learned BPE to the non-BPE tokens


Created data/maestro-v2.0.0/tokens_small_noBPE


Tokenizing MIDIs (maestro-v2.0.0/tokens_small_noBPE): 100%|██████████| 172/172 [00:21<00:00,  7.98it/s]


In [38]:
print(data_dir)

data/maestro-v2.0.0


In [48]:
import json
import numpy as np
import os

# Turns all the json files from tokenizaion in a folder into numpy arrays
def json_to_nparray(path):
    for filename in os.listdir(path):
        if filename.endswith('.json'):
            with open(path + filename) as f:
                data = json.load(f)
                np.save(path + filename[:-5], np.array(data))
            continue
        else:
            continue
json_to_nparray(path = str(path_tokens_noBPE) + "/")

In [54]:
x = path_tokens_noBPE / "tokens_array.npy"
x.mkdir(parents=True, exist_ok=True)

In [50]:
def get_tokens(path): # function to get all tokens from the dataset and put them in a numpy array
    to_shuffle = np.array(()) # to_shuffle aggregator for one piece
    data = np.array(()) # data aggregator for one piece
    piece_counter = 0

    for filename in os.listdir(path):
        if filename.endswith('.npy'):
            to_shuffle = np.append(to_shuffle, np.load(path + filename, allow_pickle=True)[()])
    np.random.shuffle(to_shuffle)

    for i in range(len(to_shuffle)):
        data = np.append(data, 1)
        for j in range(len(to_shuffle[i].get('tokens'))):
            data = np.append(data, to_shuffle[i].get('tokens')[j])
        data = np.append(data, 2)
        piece_counter += 1

    return data, piece_counter

tokens, piece_counter = get_tokens(str(path_tokens_noBPE) + "/")

np.save(path_tokens_noBPE, tokens)

TypeError: object of type 'NoneType' has no len()

In [55]:
import numpy as np
import json
from pathlib import Path

def get_tokens(path):
    piece_counter = 0
    data = np.array([], dtype=np.int64)
    json_files = list(Path(path).glob('*.json'))  # Ensure it's a list of Path objects pointing to JSON files

    to_shuffle = [json.load(open(file_path)) for file_path in json_files]  # Load JSON content from each file

    for i in range(len(to_shuffle)):
        tokens = to_shuffle[i].get('tokens')
        if tokens is not None:  # Check if 'tokens' is not None
            data = np.append(data, 1)
            for token in tokens:
                data = np.append(data, token)
            data = np.append(data, 2)
            piece_counter += 1
        else:
            print(f"No tokens in file: {json_files[i]}")  # Optionally print which file(s) are missing 'tokens'

    return data, piece_counter

# Example usage:
path_tokens_noBPE = path_tokens_noBPE  # Update this path as necessary
tokens, piece_counter = get_tokens(path_tokens_noBPE)

# Assuming you want to save 'tokens' as a NumPy array
np.save(path_tokens_noBPE / "tokens_array.npy", tokens)  # Saves the tokens array to a file named "tokens_array.npy" in the path_tokens_noBPE directory


IsADirectoryError: [Errno 21] Is a directory: 'path_tokens_noBPE/tokens_array.npy'

In [22]:
def midi_to_tokens(midi_file):
    midi_data = pretty_midi.PrettyMIDI(midi_file)
    notes = []
    for instrument in midi_data.instruments:
        for note in instrument.notes:
            notes.append((note.start, note.end, note.pitch, note.velocity))
    notes.sort(key=lambda note: note[0])  # Sort by start time
    tokens = []
    last_end_time = 0
    for note in notes:
        start_time, end_time, pitch, velocity = note
        time_shift = round(start_time - last_end_time, 2)
        if time_shift > 0:
            tokens.append(f"TimeShift_{time_shift}")
        tokens.append(f"NoteOn_{pitch}_{velocity}")
        duration = round(end_time - start_time, 2)
        tokens.append(f"Duration_{duration}")
        last_end_time = end_time
    return tokens


def process_midi_folder(folder_path):
    all_tokens = []
    for midi_file in os.listdir(folder_path):
        if midi_file.endswith(('.mid', '.midi')):
            tokens = midi_to_tokens(os.path.join(folder_path, midi_file))
            all_tokens.extend(tokens)
    return all_tokens


def create_dataset(all_tokens, seq_length=100, batch_size=64):
    # Convert tokens to IDs
    token_to_id = np.load(data_dir / "token_to_id.npy", allow_pickle=True).item()
    token_ids = [token_to_id.get(token, token_to_id['UNK']) for token in all_tokens]

    dataset = tf.data.Dataset.from_tensor_slices(token_ids)
    sequences = dataset.batch(seq_length+1, drop_remainder=True)

    def split_input_target(chunk):
        input_text = chunk[:-1]
        target_text = chunk[1:]
        return input_text, target_text

    dataset = sequences.map(split_input_target)
    dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
    return dataset

In [23]:
midi_folder_path = data_dir
#Main execution
all_tokens = process_midi_folder(midi_folder_path)


KeyboardInterrupt: 

In [8]:
from collections import Counter

# Count unique tokens and create a mapping to integers
token_counts = Counter(all_tokens)
unique_tokens = sorted(token_counts.keys())
token_to_id = {token: id for id, token in enumerate(unique_tokens, start=1)}

# Optionally, include a special token for unknown tokens (if your model needs to handle unseen tokens)
token_to_id['UNK'] = len(token_to_id) + 1

# Save the mapping for later use
np.save(data_dir / "token_to_id.npy", token_to_id)


In [ ]:
#test
file_path = data_dir / "token_to_id.npy"

# Load the mapping
token_to_idx = np.load(str(file_path), allow_pickle=True).item()

print(token_to_idx)

print(f"Number of unique tokens: {len(token_to_idx)}")



In [9]:
dataset = create_dataset(all_tokens)
all_tokens

['TimeShift_0.99',
 'NoteOn_48_43',
 'Duration_1.19',
 'NoteOn_64_52',
 'Duration_0.8',
 'NoteOn_60_49',
 'Duration_0.77',
 'NoteOn_55_38',
 'Duration_1.16',
 'NoteOn_59_56',
 'Duration_0.05',
 'NoteOn_62_62',
 'Duration_0.06',
 'TimeShift_0.03',
 'NoteOn_60_60',
 'Duration_0.05',
 'TimeShift_0.02',
 'NoteOn_59_58',
 'Duration_0.05',
 'NoteOn_62_51',
 'Duration_0.05',
 'TimeShift_0.05',
 'NoteOn_60_58',
 'Duration_0.04',
 'NoteOn_64_37',
 'Duration_0.04',
 'TimeShift_0.01',
 'NoteOn_65_67',
 'Duration_0.04',
 'NoteOn_62_65',
 'Duration_0.04',
 'TimeShift_0.14',
 'NoteOn_64_64',
 'Duration_0.04',
 'NoteOn_60_47',
 'Duration_0.03',
 'TimeShift_0.17',
 'NoteOn_62_68',
 'Duration_0.05',
 'NoteOn_59_58',
 'Duration_0.05',
 'NoteOn_55_54',
 'Duration_0.05',
 'NoteOn_43_54',
 'Duration_0.05',
 'TimeShift_0.35',
 'NoteOn_67_65',
 'Duration_0.04',
 'NoteOn_59_60',
 'Duration_0.04',
 'NoteOn_55_49',
 'Duration_0.06',
 'NoteOn_43_46',
 'Duration_0.05',
 'TimeShift_1.28',
 'NoteOn_43_47',
 'Durati

In [10]:
#Create the Positional Encodings which are added to the Embeddings
def pos_enc(length, d_model):
    pos = np.arange(length)[:, np.newaxis]
    j = np.arange(d_model)[np.newaxis, :]
    #distribute the angles according to the formula for Positional Encodings
    angle_rates = 1 / np.power(10000, (2 * (j//2)) / np.float32(d_model))
    angle_rads = pos * angle_rates
    #even coordinates get sin, odd coordinates get cos
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[np.newaxis, ...]
    return tf.cast(pos_encoding, dtype=tf.float32)

The formulas for Positional Encodings from the original Attention Paper are $$PE_{(pos, 2j)} = \sin(pos / 10000^{2j / d_{\text{model}}})$$
$$PE_{(pos, 2j+1)} = \cos(pos / 10000^{2j / d_{\text{model}}})
$$ so we have to implement two formulas, one for the even and one for the odd coordinates.

In [11]:
import tensorflow as tf

#Create the Embeddings our Transformer can use
class Embedding(tf.keras.layers.Layer):
    def __init__(self, voc, d_model, block_length, dropout_rate):
        super(Embedding, self).__init__()
        self.voc = voc
        self.block_length = block_length
        self.d_model = d_model
        self.emb = tf.keras.layers.Embedding(input_dim=voc,
                          output_dim=d_model,
                          input_length=block_length)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.pos_enc = pos_enc(block_length, d_model)

    #Add the Embeddings and the Encodings
    @tf.function
    def call(self, x):
        x = self.emb(x) + self.pos_enc[:, :self.block_length, :]
        x = self.dropout(x)
        return x

In [12]:
#SwigLu
class SwiGLu(tf.keras.layers.Layer):
    def __init__(self, d_model, units):
        super(SwiGLu, self).__init__()
        self.dense1 = tf.keras.layers.Dense(units=(units//3)*2, activation=tf.nn.silu, use_bias=False)
        self.dense2 = tf.keras.layers.Dense(units=(units//3)*2, use_bias=False)
        self.dense3 = tf.keras.layers.Dense(units=d_model, use_bias=False)

    @tf.function
    def call(self, x):
        w = self.dense1(x)
        v = self.dense2(x)
        x = self.dense3(w*v)
        return x

SwigLu is a special type of Activation Function, where the name Activation Function is actually a bit misleading, because it consists of multiple NN Layers. At first, the input is put into two different Dense Layers having $\frac{2}{3} \cdot d_{model}$ units. The outputs of those layers are then multiplied element-wise, and the result is put into a final Dense Layers, so the output dimensionality is $d_{model}$ again. The SiLu Activation Function in the first Dense Layer is defined as $SiLu(x) = x \cdot Sigmoid(x)$.

In [13]:
#Implement the classic attention layer
class Attention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dropout_rate):
        super(Attention, self).__init__()
        #Choose head_size in a way that the total dimensionality does not change
        self.size_heads = d_model // num_heads
        self.query = tf.keras.layers.Dense(units=self.size_heads, use_bias=False)
        self.key = tf.keras.layers.Dense(units=self.size_heads, use_bias=False)
        self.value = tf.keras.layers.Dense(units=self.size_heads, use_bias=False)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)

    #Implement the Attention function
    @tf.function
    def call(self, x):
        B, T, C = x.shape
        Q = self.query(x)
        K = self.key(x)
        V = self.value(x)
        #Compute the Attention matrix
        scores = tf.matmul(Q,tf.transpose(K, perm=[0, 2, 1])) / tf.math.sqrt(tf.cast(self.size_heads, tf.float32))
        #Mask for training
        tril   = tf.linalg.band_part(tf.ones((T, T)), -1, 0)
        scores = tf.where(tril == 0, tf.fill(tril.shape, -float('inf')), scores)
        #Continue Computing the Attention Values
        scores = tf.nn.softmax(scores, axis=-1)
        scores = self.dropout(scores)
        return tf.matmul(scores, V)

The Attention formula is $\text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$, where the softmax is applied to each row of the matrix. First, we compute $\frac{QK^T}{\sqrt{d_k}}$, then after masking, we put it into the softmax, apply dropout and multiply by V.


In [14]:
#Implement Multi-Head-Attention
class MultiHead(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dropout_rate):
        super(MultiHead, self).__init__()
        self.heads = [Attention(d_model, num_heads, dropout_rate) for _ in range(num_heads)]
        self.dense = tf.keras.layers.Dense(units=d_model, use_bias=False)
    #Write the results in one concatenated matrix to keep the dimensionality
    @tf.function
    def call(self, x):
        x = tf.concat([head(x) for head in self.heads], axis=-1)
        x = self.dense(x)
        return x

In [15]:
#Transformer block
class Tblock(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, units, dropout_rate):
        super(Tblock, self).__init__()
        self.ffn = SwiGLu(d_model, units)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.attention = MultiHead(d_model, num_heads, dropout_rate)
        self.layernorm1 = tf.keras.layers.LayerNormalization()
        self.layernorm2 = tf.keras.layers.LayerNormalization()
        self.add = tf.keras.layers.Add()

    @tf.function
    def call(self, x):
        res = x
        x = self.layernorm1(x)
        x = self.attention(x)
        x = self.dropout(x)
        x = self.add([res, x])
        res = x
        x = self.layernorm2(x)
        x = self.ffn(x)
        x = self.dropout(x)
        x = self.add([res, x])
        return x


In [16]:
#Decoder
class Decoder(tf.keras.layers.Layer):
    def __init__(self, voc, d_model, num_heads, units, dropout_rate, num_layers, block):
        super(Decoder, self).__init__()
        self.embedding = Embedding(voc, d_model, block, dropout_rate)
        self.layers = [Tblock(d_model, num_heads, units, dropout_rate) for _ in range(num_layers)]


    @tf.function
    def call(self, x):
        x = self.embedding(x)
        for layer in self.layers:
            x = layer(x)
        return x

In [17]:
#Full Transformer
class Transformer(tf.keras.Model):
    def __init__(self, voc, d_model, num_heads, units, dropout_rate, num_layers, block):
        super(Transformer, self).__init__()
        self.decoder = Decoder(voc, d_model, num_heads, units, dropout_rate, num_layers, block)
        self.final_dense = tf.keras.layers.Dense(units=voc)

    @tf.function
    def call(self, x):
        x = self.decoder(x)
        logits = self.final_dense(x)
        return logits

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super().__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    step = tf.cast(step, dtype=tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [18]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()
        self.d_model = tf.cast(d_model, tf.float32)  # Ensure d_model is a float
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, tf.float32)  # Cast step to float to avoid type issues
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)



In [ ]:
import tensorflow as tf


transformer = Transformer(
    voc=1024,
    d_model=512,
    num_heads=8,
    units=2048,
    dropout_rate=0.1,
    num_layers=6,
    block=100
)

learning_rate = CustomSchedule(d_model=512)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

#Loss
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

#Training Step
train_loss = tf.keras.metrics.Mean(name='train_loss')

@tf.function
def train_step(inp, tar):
    with tf.GradientTape() as tape:
        predictions = transformer(inp, training=True)
        loss = loss_function(tar, predictions)

    gradients = tape.gradient(loss, transformer.trainable_variables)
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

    train_loss(loss)




Epoch 1, Loss: nan
Epoch 2, Loss: nan
Epoch 3, Loss: nan
Epoch 4, Loss: nan
Epoch 5, Loss: nan
Epoch 6, Loss: nan
Epoch 7, Loss: nan
Epoch 8, Loss: nan
Epoch 9, Loss: nan
Epoch 10, Loss: nan
Epoch 11, Loss: nan


In [ ]:
#Training loop
EPOCHS = 20

for epoch in range(EPOCHS):
    for (batch, (inp, tar)) in enumerate(dataset):
        train_step(inp, tar)

    print(f'Epoch {epoch + 1}, Loss: {train_loss.result()}')
    train_loss.reset_states()